Niniejszy arkusz pokazuje zastosowanie modułu JamSpell w kwestii sprawdzania pisowni.
Jamspell umożliwia sprawdzanie pisowni, nie zawiera jednak słownika języka polskiego,
dlatego musiałem utworzyć własny słownik (a raczej dwa dla porównania).

Jeden ze słowników do tekst z 1000 polskich nowel, jakie są udostępione na stronie CLARIN (https://clarin-pl.eu/dspace/handle/11321/312). Drugi słownik to dump z: 300K losowych zdać z polskich stron (2018 rok), 100K zdań z kanałów RSS z wiadomościami (2020 rok), 300K zdań z innych serwisów informacyjncyh (2018 rok) oraz dump 300K zdań z polskiej Wikipedii z 2021 roku.

Każdy z ww. tekstów został uzyskany w postaci osobnego pliku tekstowego. Wszystkie te pliki połączyłem do jednego pliku (kodowanie utf-8) za pomocą poniższego kodu:


In [ ]:
%%writefile combine.py

import glob

read_files = glob.glob("*.txt")
total_len = len(read_files)

count = 1
with open("__result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())
            print(f'{count}/{total_len}')
            count +=1

Plik `combine.py` umieściłem i uruchomiłem w folderze z plikami txt, aby je połączyć. Utworzyłem dodatkowo prosty plik tekstowy `alphabet_pl.txt` z możliwymi polskimi znakami (same małe litery) `abcdefghijklmnoprstuwyząćęłńóśżź`. Za pomocą skryptu `combine.py` w sumie utworzyłem dwa pliki `__result.txt` (nowele) i `__result_dumps.txt` (dump z polskiego internetu).

Github https://github.com/bakwc/JamSpell zawiera instrukcję jak sklonować i utworzyć JamSpell.
Moja rada: używać tutaj bezwzględnie Linuxa! Pod Windowsem mimo zainstalowanego MinGW i ustawionych ścieżek w zmiennych systemowych, cały czas pojawiały się problemy, najpierw z nmake, potem z cmake, których już oficjalnie nie można nigdzie ściągnąć i należałoby doinstalować jakieć kilka GB pakietów z Visual Studio, żeby tylko dostać jeden plik nmake.exe i to też bez gwarancji, że coś się nie wykrzaczy po drodze. 
Ostatecznie przeczuciłem się na Ubuntu i build JamSpell poszedł bez problemów.
Pliki `__result.txt`, `__result_dumps.txt` i `alphabet_pl.txt` umieściłem w katalogu `test_data` JamSpella.

Po zbudowaniu JamSpell i wejściu do katalogu build, słowniki utworzyłem za pomocą poleceń:

` ./main/jamspell train ../test_data/alphabet_pl.txt ../test_data/__result.txt spell_polish_novels.bin `

oraz

` ./main/jamspell train ../test_data/alphabet_pl.txt ../test_data/__result_dumps.txt spell_polish_dumps.bin `


Mając już utworzone dwa pliki `.bin` używam ich do sprawdzenia polskiej pisowni dla naszego zbioru. Sprawdzenie pisowni wykonuję na Colabie.

In [ ]:
# upload potrzebnych plików na Colaba
from google.colab import files
uploaded = files.upload()

In [ ]:
# swig jest wymagane dla Jamspell
!sudo apt-get install swig

In [ ]:
# Instalacja Jamspell
!sudo pip install jamspell

In [ ]:
# Upgrade tables, bez tego nie otworzę i nie zapiszę formatu HDF (zwróci błąd)
pip install --upgrade tables

In [8]:
# Załadowanie bibliotek
import pandas as pd
import jamspell

In [9]:
# Załadowanie zbioru
df = pd.read_hdf('./outputs/budzety_spellchecked.h5')

In [13]:
# 10 losowych pozycji
df.sample(10)

,nazwa,correct_Speller
7032,OSWIETLAMY NOWY KONIN OSWIETLENIE PARKINGU POM...,OSWIETLAMY OW ONI OSWIETLENIE PARKINGU POMIEDZ...
4131,L16/15/VI - 100-lecie klubu NAPRZÓD JANÓW - ws...,L16/15/VI - 100-lecie klubu NARÓD JAN - wspóln...
2085,21. Ustawienie sprzętu sportowego na placu zab...,21. Ustawienie sprzętu sportowego na placu zab...
22122,Spotkania z muzyką klasyczną,Spotkania z muzyką klasyczną
2952,Droga na Pogorię,Droga na Pogoria
21618,"Joga, taniec, medytacja, w otwartych przestrze...","Joga, taniec, medytacja, w otwartych przestrze..."
12902,Porządkowanie osiedlowej przestrzeni - budowa ...,Porządkowanie osiedlowej przestrzeni - budowa ...
23494,Park tokietka,Park lokietka
19215,Gocławskie Centrum Tańca,Wrocławskie Centrum Tańca
6419,152. Energia dla Olsztyna,152. Energia dla Olsztyna


In [14]:
# Najpierw zabieramy się za dump z polskiego internetu
jsp = jamspell.TSpellCorrector()
assert jsp.LoadLangModel('./models/spell_polish_dumps.bin')

In [19]:
# Sprawdzamy czy działa
jsp.FixFragment('To jes źlle napisane ttumaczenie')

'To jest źle napisane tłumaczenie'

In [20]:
# Uruchamiany sprawdzanie pisowni dla naszego zbioru
df['correct_jamspell_dumps'] = df['nazwa'].apply(jsp.FixFragment)

In [21]:
df

,nazwa,correct_Speller,correct_jamspell_dumps
0,Projektistatus,Projektistatus,Projektistatus
1,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...
2,Cykliczne warsztaty dla seniorów z dzielnicy O...,Cykliczne warsztaty dla seniorów z dzielnicy O...,Cykliczne warsztaty dla seniorów z dzielnicy O...
3,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana","Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana","Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana"
4,Montaż słupów oświetleniowych na chodniku przy...,Montaż słupów oświetleniowych na chodniku przy...,Montaż słupów oświetleniowych na chodniku przy...
...,...,...,...
23687,Zielony skwer na Zabtociu,Zielony skwer na Zabłociu,Zielony skwer na Zabłociu
23688,Znani Biatopradniczanie dla Mieszkanców,Znani Biatopradniczanie dla Mieszkańców,Znani Biatopradniczanie dla Mieszkanców
23690,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultu...,ZRÓB WER RZ FABRYCZNEJ! Edukacja i kultura w z...,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultu...
23691,Zróbmy sobie Park,Zróbmy sobie Park,Zróbmy sobie Park


In [24]:
# Teraz polskie nowele
assert jsp.LoadLangModel('./models/spell_polish_novels.bin')

In [25]:
df['correct_jamspell_novels'] = df['nazwa'].apply(jsp.FixFragment)

In [38]:
df.sample(20)

,nazwa,correct_Speller,correct_jamspell_dumps,correct_jamspell_novels
14280,MAMY przestrzeń w Parku Poniatowskiego.,AMY przestrzeń w Parku Poniatowskiego.,MAMY przestrzeń w Parku Poniatowskiego.,MAMY przestrzeń w Parku Poniatowskiego.
14189,"Renowacja pomnika ""Macierzyństwo"".","Renowacja pomnika ""Macierzyństwo"".","Renowacja pomnika ""Macierzyństwo"".","Rewelacja pomnika ""Macierzyństwo""."
22757,Warsztaty śpiewu dla dzieci i dorosłych,Warsztaty śpiewu dla dzieci i dorosłych,Warsztaty śpiewu dla dzieci i dorosłych,Warsztaty śpiewu dla dzieci i dorosłych
6136,3 Przygotowanie do uzytkowania czerwonego szla...,3 Przygotowanie do uzytkowania czerwonego szla...,3 Przygotowanie do użytkowania czerwonego szla...,3 Przygotowanie do uzytkowania czerwonego szla...
18343,Linarium – plac zabaw dla młodzieży (ul. Kierz...,Luminarium – plac zabaw dla młodzieży (ul. Kie...,Solarium – plac zabaw dla młodzieży (ul. Kierz...,Delirium – plac zabaw dla młodzieży (ul. Skier...
14954,"Parki przyjazne jeżom, owadom, ptakom...","Parki przyjazne jeżom, owadom, ptakom...","Parki przyjazne jeżom, owadom, ptakom...","Parki przyjazne jenom, owadom, ptakom..."
7559,Samoobsługowe stacje naprawy rowerów,Samoobsługowe stacje naprawy rowerów,Samoobsługowe stacje naprawy rowerów,Samoobsługowe stacje naprawy rowerów
10897,Tor samochodowych modeli RC,Tor samochodowych modeli RC,Tor samochodowych modeli RC,Tor samochodowych modeli RC
6313,42. EKO STACJA POJEZIERZE,42. KO TACA POJEZIERZE,42. EKO STACJA POJEZIERZE,42. OKO STACJA POJEZIERZE
1921,12. Kontynuacja zagospodarowania placu przy ul...,12. Kontynuacja zagospodarowania placu przy ul...,12. Kontynuacja zagospodarowania placu przy ul...,12. Kontynuacja zagospodarowania placu przy ul...


In [30]:
# Zapisuję do pliku do późniejszej analizy
df.to_hdf('budzety_spellchecked_JamSpell.h5', key='df')